In [2]:
#https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
import io
application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))
# application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifications_to_replace = []
for el in application_df['CLASSIFICATION'].unique():
  if application_df.CLASSIFICATION.value_counts()[el] > 50:
    pass
  else:
    classifications_to_replace.append(el)
print(len(classifications_to_replace))

57


In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      441
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(application_df, drop_first=True)
df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1200,CLASSIFICATION_C1270,CLASSIFICATION_C1300,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2700,CLASSIFICATION_C2800,CLASSIFICATION_C3000,CLASSIFICATION_C4000,CLASSIFICATION_C5000,CLASSIFICATION_C7000,CLASSIFICATION_C7100,CLASSIFICATION_Other,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0


In [9]:
# Split our preprocessed data into our features and target arrays
y = df['IS_SUCCESSFUL']
df = df.drop(columns=['IS_SUCCESSFUL'])
df.head()
# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split

train, test, y_train, y_test = train_test_split(df, y, test_size=0.2)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(train)

# Scale the data
X_train_scaled = X_scaler.transform(train)
X_test_scaled = X_scaler.transform(test)

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# FIRST OPTIMIZATION ATTEMPT = CHANGE ACTIVATION FUNCTIONS FOR INPUT AND SECOND LAYERS FROM RELU TO TANH AND REDUCE TOTAL EPOCHS

nn_model = tf.keras.models.Sequential()

# first hidden layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="tanh", input_dim=46))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="tanh"))

# output layer 
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 282       
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 331
Trainable params: 331
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=125)

Epoch 1/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5952 - accuracy: 0.6998
Epoch 2/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5634 - accuracy: 0.7275
Epoch 3/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5582 - accuracy: 0.7285
Epoch 4/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5556 - accuracy: 0.7293
Epoch 5/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5539 - accuracy: 0.7303
Epoch 6/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5528 - accuracy: 0.7299
Epoch 7/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5520 - accuracy: 0.7311
Epoch 8/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.7307
Epoch 9/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5508 - accuracy: 0.7306
Epoch 10/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5506 - accura

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5498 - accuracy: 0.7223 - 354ms/epoch - 2ms/step
Loss: 0.5497931241989136, Accuracy: 0.7223032116889954


In [15]:
# First optimization attempt did slightly better on the test data, so we will keep the changes going into second attempt.

In [16]:
nn_model = tf.keras.models.Sequential()

# SECOND OPTIMIZATION ATTEMPT NOW INCREASES NUMBER OF NEURONS IN FIRST TWO LAYERS
# first hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="tanh", input_dim=46))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="tanh"))

# output layer 
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 376       
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 457
Trainable params: 457
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=125)

Epoch 1/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5926 - accuracy: 0.7017
Epoch 2/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5660 - accuracy: 0.7238
Epoch 3/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5600 - accuracy: 0.7268
Epoch 4/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5568 - accuracy: 0.7285
Epoch 5/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5542 - accuracy: 0.7281
Epoch 6/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5528 - accuracy: 0.7302
Epoch 7/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7314
Epoch 8/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5509 - accuracy: 0.7313
Epoch 9/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5500 - accuracy: 0.7320
Epoch 10/125
858/858 [==============================] - 1s 2ms/step - loss: 0.5493 - accura

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5499 - accuracy: 0.7293 - 337ms/epoch - 2ms/step
Loss: 0.5499154925346375, Accuracy: 0.729300320148468


In [32]:
nn_model = tf.keras.models.Sequential()
# Second optimization attempt again slightly increased accuracy of model on test data, so I will carry forward changes.
# THIRD OPTIMIZATION ATTEMPT NOW INCREASES NUMBER OF NEURONS IN FIRST TWO LAYERS EVEN MORE, AND ADDS ADDITIONAL HIDDEN LAYER. 
# first hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="tanh", input_dim=46))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="tanh"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="tanh"))


# output layer 
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 10)                470       
                                                                 
 dense_21 (Dense)            (None, 10)                110       
                                                                 
 dense_22 (Dense)            (None, 10)                110       
                                                                 
 dense_23 (Dense)            (None, 1)                 11        
                                                                 
Total params: 701
Trainable params: 701
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=125)

Epoch 1/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5887 - accuracy: 0.7067
Epoch 2/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5624 - accuracy: 0.7244
Epoch 3/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7292
Epoch 4/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5526 - accuracy: 0.7299
Epoch 5/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7318
Epoch 6/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7316
Epoch 7/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7319
Epoch 8/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7329
Epoch 9/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7328
Epoch 10/125
858/858 [==============================] - 2s 2ms/step - loss: 0.5467 - accura

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5487 - accuracy: 0.7316 - 350ms/epoch - 2ms/step
Loss: 0.5486839413642883, Accuracy: 0.731632649898529


In [36]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimization.h5")